# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [3]:
env = UnityEnvironment(file_name='Reacher_Linux/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.06649999851360917


When finished, you can close the environment.

In [6]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [15]:
LR = 5e-4
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)
        
class Actor(nn.Module):
    def __init__(self, state_size, action_size, fc_sizes=[128,64,32]):
        super(Actor, self).__init__()
        sequence_dict = OrderedDict();
        sequence_dict['fc0'] = nn.Linear(state_size, fc_sizes[0])
        sequence_dict['fc_relu0'] = nn.ReLU()
        for i,fc_size in enumerate(fc_sizes):
            if(i == len(fc_sizes)-1):
                sequence_dict['logit'] = nn.Linear(fc_size, action_size)
            else:
                sequence_dict['fc{}'.format(i+1)] = nn.Linear(fc_size, fc_sizes[i+1])
                sequence_dict['fc_relu{}'.format(i+1)] = nn.ReLU()
        sequence_dict['sigmoid'] = nn.Sigmoid()
        self.model = nn.Sequential(sequence_dict)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=LR)
        self.model.apply(init_weights)

    def forward(self, state):
        res = self.model.forward(state)
        return  torch.log(res), (res-0.5)*2.0
    
class Critic(nn.Module):
    def __init__(self, state_size, fc_sizes=[128,64,32]):
        super(Critic, self).__init__()
        sequence_dict = OrderedDict();
        sequence_dict['fc0'] = nn.Linear(state_size, fc_sizes[0])
        sequence_dict['fc_relu0'] = nn.ReLU()
        for i,fc_size in enumerate(fc_sizes):
            if(i == len(fc_sizes)-1):
                sequence_dict['logit'] = nn.Linear(fc_size, 1)
            else:
                sequence_dict['fc{}'.format(i+1)] = nn.Linear(fc_size, fc_sizes[i+1])
                sequence_dict['fc_relu{}'.format(i+1)] = nn.ReLU()
        sequence_dict['tanh'] = nn.Tanh()
        self.model = nn.Sequential(sequence_dict)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=LR)
        self.model.apply(init_weights)

    def forward(self, state):
        return self.model.forward(state)

actor = Actor(state_size,action_size, fc_sizes=[256,128,64,32]).to(device)
critic = Critic(state_size,  fc_sizes=[64,32]).to(device)

In [16]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations

In [17]:
state = torch.from_numpy(states).float().to(device)
log_actions, actions = actor(state)
critic(state)

tensor([[-0.9725],
        [-0.5720],
        [-0.9972],
        [-0.9874],
        [-0.6327],
        [-0.6091],
        [-0.9981],
        [-0.9970],
        [-0.9952],
        [-0.9951],
        [-0.9950],
        [-0.9939],
        [-0.9794],
        [-0.9638],
        [-0.9187],
        [-0.9406],
        [-0.9980],
        [-0.9938],
        [-0.9953],
        [-0.9605]], device='cuda:0', grad_fn=<TanhBackward>)

## 5.Training loop 

Here we train the policy.

In [ ]:
n_episodes = 1000
max_steps = 500
gamma = 0.95
score_array = []
zero_counter = 0
for e in range(n_episodes):
    env_info = env.reset(train_mode=True)[brain_name]
    states = env_info.vector_observations
    scores = np.zeros(num_agents)
    rollout = []
    #acquiring an episode
    for t in range(max_steps):
        states = torch.from_numpy(states).float().to(device)        
        rewards = env_info.rewards
        rewards = np.array(rewards)
        rewards = rewards.reshape(rewards.shape[0],1)
        rewards = torch.from_numpy(rewards).float().to(device)
        dones = env_info.local_done
        dones = np.array(dones, dtype=np.float)
        dones = dones.reshape(dones.shape[0],1)
        dones = torch.from_numpy(dones).float().to(device)
        
        #using actor to get action to take the next action
        log_actions, actions = actor(states)
        
        env_info = env.step(actions.cpu().data.numpy())[brain_name]
        next_states = env_info.vector_observations
        
        values_s = critic(states)
        future_rewards = critic(torch.from_numpy(next_states).float().to(device))
        future_rewards = rewards + gamma*future_rewards*dones
        
        rollout.append((log_actions, future_rewards, values_s, dones))
        scores += env_info.rewards
        states = next_states
        if np.all(dones):
            break
            
    # Now updating the weights
    for i in reversed(range(len(rollout) - 1)):
        log_actions, future_rewards, values_s, dones = rollout[i]

        #update the actor
        #try to use the values before updated critic first
        advantage = (future_rewards - values_s)*dones
        loss_actor = (-log_actions * advantage).mean()+0.00001
        actor.optimizer.zero_grad()
        loss_actor.backward(retain_graph=True)
        actor.optimizer.step()
        
        #update the critic
        loss_critic = (0.5*(values_s - future_rewards)**2).sum()+0.00001
        critic.optimizer.zero_grad()
        loss_critic.backward(retain_graph=True)
        critic.optimizer.step()
    
    score_mean = np.mean(scores)
    score_array.append(score_mean)
    print("Now on epoch: {} with score: {:.2f}".format(e, score_mean))
    if(e%10 == 0):
        torch.save(actor.model.state_dict(), 'actor.pth')
        torch.save(critic.model.state_dict(), 'critic.pth')
    if(score_mean <= 0.01):
        zero_counter += 1
        if(zero_counter == 10):
            break
    else:
        zero_counter = 0

Now on epoch: 0 with score: 0.34
Now on epoch: 1 with score: 0.32
Now on epoch: 2 with score: 0.37
Now on epoch: 3 with score: 0.40
Now on epoch: 4 with score: 0.34
Now on epoch: 5 with score: 0.40
Now on epoch: 6 with score: 0.42
Now on epoch: 7 with score: 0.39
Now on epoch: 8 with score: 0.39
Now on epoch: 9 with score: 0.36
Now on epoch: 10 with score: 0.29
Now on epoch: 11 with score: 0.45
Now on epoch: 12 with score: 0.40
Now on epoch: 13 with score: 0.46
Now on epoch: 14 with score: 0.44
Now on epoch: 15 with score: 0.41
Now on epoch: 16 with score: 0.42
Now on epoch: 17 with score: 0.40
Now on epoch: 18 with score: 0.41
Now on epoch: 19 with score: 0.44
Now on epoch: 20 with score: 0.47
Now on epoch: 21 with score: 0.41
Now on epoch: 22 with score: 0.50
Now on epoch: 23 with score: 0.39
Now on epoch: 24 with score: 0.39
Now on epoch: 25 with score: 0.43
Now on epoch: 26 with score: 0.36
Now on epoch: 27 with score: 0.36
Now on epoch: 28 with score: 0.48
Now on epoch: 29 with sc

In [77]:
future_rewards

tensor([[-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500],
        [-0.9500]], device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(score_array)), score_array)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [97]:
scores

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.11, 0.  ,
       0.  , 0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])